In [1]:
import numpy as np
import ipywidgets as widgets
import pandas as pd
import altair as alt
from datetime import datetime
import random
from matplotlib import pyplot as plt
pd.set_option('display.max_columns', None)
crime_df = pd.read_csv("https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv")
crime_df["DATE  OF OCCURRENCE"] = pd.to_datetime(crime_df["DATE  OF OCCURRENCE"])
from datetime import datetime
crime_df["dayname"]= crime_df["DATE  OF OCCURRENCE"].apply(lambda x: x.strftime('%A'))
crime_df["Hour"] = crime_df["DATE  OF OCCURRENCE"].apply(lambda x : x.strftime("%H"))

In [2]:
crime_df["Month"] = crime_df["DATE  OF OCCURRENCE"].apply(lambda x : x.strftime("%m"))

In [5]:
color_scale = alt.Scale(domain=list(crime_df[" PRIMARY DESCRIPTION"].value_counts().index),
                        range=['#'+"%06x" % random.randint(1, 0xFFFFFF) for i in range(1,32)])
color_scale1 = alt.Scale(domain=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],
                        range=['#'+"%06x" % random.randint(0, 0xFFFFFF) for i in range(1,13)])
# interval selection in the scatter plot

pts1 = alt.selection(type="interval", encodings=["x"])
# left panel: scatter plot
mag1 = alt.Chart().mark_bar().encode(
    x='month(DATE  OF OCCURRENCE):O',
    y="count()",
    color = alt.Color('month(DATE  OF OCCURRENCE):O',scale = color_scale1),
#     color=alt.condition(pts, alt.value("black"), alt.value("lightgray"))
).properties(
    width=300,
    height=300
).add_selection(pts1)

# right panel: histogram
mag2 = alt.Chart().mark_line(point = True).encode(
    x='hours(DATE  OF OCCURRENCE):O',
    y="count()",
#     color=alt.condition(pts, alt.value("black"), alt.value("lightgray"))
).transform_filter(
    pts1
).properties(
    width=300,
    height=300
)

mag3 = alt.Chart().mark_bar().encode(
    x=' PRIMARY DESCRIPTION:O',
    y="count()",
    color = alt.Color(' PRIMARY DESCRIPTION:O',scale = color_scale)
#     color=alt.condition(pts, alt.value("black"), alt.value("lightgray"))
).transform_filter(
    pts1
).properties(
    width=700,
    height=300
)

# build the chart:
up = alt.hconcat(
    mag1,
    mag2,
    data="https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv"
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=20)
)
alt.vconcat(
    up,
    mag3,
    data="https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv"
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=20)
)


alt.VConcatChart(...)

In [6]:
chart = alt.vconcat(
    up,
    mag3,
    data="https://data.cityofchicago.org/api/views/x2n5-8w5q/rows.csv"
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=20)
)
chart.save('chart.json')